Route Between Nodes: Given a directed graph, design an algorithm to find out whether there is a
route between two nodes.

## Without libraries

### Bidirectional search

In [12]:
s1 = {1, 2}
s2 = {2, 3}

s1.intersection(s2)

{2}

In [15]:
from collections import defaultdict
from collections import deque
from pprint import pprint
from itertools import zip_longest

# Graph:
# 1__{2, 10, 20}
# |
# 3__4__5__6__7
#
# X__Y__Z ( ---> ) 1
# |
# W

edges = [
    (1, 11), (1, 12), (1, 13), (1, 14), (1, 15),
    (12, 23), (23, 34), (34, 45),
    (45, 1), # Cycle!
    ("A", "B"), ("B", "C"), ("C", "D"),
    ("A", "A1"),
    ("A", "A2"),
    ("A", "A3"), ("A3", "A31"),
    ("D", 23),

]
graph = defaultdict(list)  # Implement as a hash table of adjacencies per node

for a, b in edges:
    graph[a].append(b)

pprint(graph)

####

a, b = 1, "A"
####

a_links = graph[a]
b_links = graph[b]

# BFS
queue = deque()
marked_a = set()

queue_b = deque()
marked_b = set()

queue_a.append(a)
marked_a.add(a)

queue_b.append(b)
marked_b.add(b)

while len(queue_a) or len(queue_b):
    if len(queue_a):
        node_a = queue_a.popleft()
        marked_a.add(node_a)

    if len(queue_b):
        node_b = queue_b.popleft()
        marked_b.add(node_b)

    if marked_a.intersection(marked_b):
        print("PATH FOUND")
        break

    neighbors_a = graph.get(node_a, [])
    neighbors_b = graph.get(node_b, [])

    for neighbor_a, neighbor_b in zip_longest(neighbors_a, neighbors_b):
        if neighbor_a and neighbor_a not in marked_a:
            queue_a.append(neighbor_a)
        if neighbor_b and neighbor_b not in marked_b:
            queue_b.append(neighbor_b)

    print(f"{queue_a = }, {marked_a = }")
    print(f"{queue_b = }, {marked_b = }")
    print("---")

defaultdict(<class 'list'>,
            {1: [11, 12, 13, 14, 15],
             12: [23],
             23: [34],
             34: [45],
             45: [1],
             'A': ['B', 'A1', 'A2', 'A3'],
             'A3': ['A31'],
             'B': ['C'],
             'C': ['D'],
             'D': [23]})
queue_a = deque([11, 12, 13, 14, 15]), marked_a = {1}
queue_b = deque(['B', 'A1', 'A2', 'A3']), marked_b = {'A'}
---
queue_a = deque([12, 13, 14, 15]), marked_a = {1, 11}
queue_b = deque(['A1', 'A2', 'A3', 'C']), marked_b = {'B', 'A'}
---
queue_a = deque([13, 14, 15, 23]), marked_a = {1, 11, 12}
queue_b = deque(['A2', 'A3', 'C']), marked_b = {'B', 'A1', 'A'}
---
queue_a = deque([14, 15, 23]), marked_a = {1, 11, 12, 13}
queue_b = deque(['A3', 'C']), marked_b = {'B', 'A2', 'A1', 'A'}
---
queue_a = deque([15, 23]), marked_a = {1, 11, 12, 13, 14}
queue_b = deque(['C', 'A31']), marked_b = {'A2', 'A1', 'B', 'A', 'A3'}
---
queue_a = deque([23]), marked_a = {1, 11, 12, 13, 14, 15}
queue_b = deque

##  Second take at Bidirectional BFS -- Only one Queue

In [24]:
from collections import defaultdict


edges = [
    (1, 11), (1, 12), (1, 13), (1, 14), (1, 15),
    (12, 23), (23, 34), (34, 45),
    (45, 1), # Cycle!
    ("A", "B"), ("B", "C"), ("C", "D"),
    ("A", "A1"),
    ("A", "A2"),
    ("A", "A3"), ("A3", "A31"),
    ("D", 23),

]
graph = defaultdict(list)
for a, b in edges:
    graph[a].append(b)

In [27]:
from collections import deque


start, end = 1, "A"
####

# Worst case: O(N + M)
# If there is a route: O(K^{d/2}) where K is avg connections, d degree of sep

def is_route_bidirectional(graph: dict, start, end) -> bool:
    # TODO: Docstring
    to_visit = deque()
    to_visit.append(start)
    to_visit.append(end)
    visited_start = {start}
    visited_end = {end}

    while to_visit:
        parent = to_visit.popleft()

        if parent in visited_start and parent in visited_end:
            return True

        for child in graph[parent]:
            if parent in visited_start and child not in visited_start:
                to_visit.append(child)
                visited_start.add(child)

            if parent in visited_end and child not in visited_end:
                to_visit.append(child)
                visited_end.add(child)

    return False

Visiting: 1, Children: [11, 12, 13, 14, 15]
Visiting: A, Children: ['B', 'A1', 'A2', 'A3']
Visiting: 11, Children: []
Visiting: 12, Children: [23]
Visiting: 13, Children: []
Visiting: 14, Children: []
Visiting: 15, Children: []
Visiting: B, Children: ['C']
Visiting: A1, Children: []
Visiting: A2, Children: []
Visiting: A3, Children: ['A31']
Visiting: 23, Children: [34]
Visiting: C, Children: ['D']
Visiting: A31, Children: []
Visiting: 34, Children: [45]
Visiting: D, Children: [23]
Visiting: 45, Children: [1]
Visiting: 23, Children: [34]
A path has been found!


## Gayle 1: BFS

In [50]:
from enum import Enum
from collections import deque


graph = {
    1: [2],
    2: [3],
    3: [4],
    4: [1, 5],
    5: [6, 7, 8, 9, 10, 11],
}

def get_children(node, graph):
    return graph.get(node, [])

def is_route_BFS(start, end, graph):
    if start == end:
        return True
    
    to_visit = deque([start])
    visited = {start}

    while to_visit:
        node = to_visit.popleft()
        for child in get_children(node, graph):
            if child in visited:
                continue
            if child == end:
                return True
            to_visit.append(child)
            visited.add(child)

    return False


is_route_BFS(1, 0, graph)

False